**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
# skvideo.setFFmpegPath(r"C:\Users\Deepesh\Downloads\ffmpeg-4.2.2-win64-shared\bin")
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The `act` function of the class `Agent` takes the present state as input and aims to decide what step to take next. 

In the beginning stages of the game, the agent does not have enough prior knowledge of the game. It is something that the agent needs to learn over time and in order to **learn**, it needs to explore various unseen states of the game. This is the classic exploration/exploitation aspect of Reinforcement Learning. The agent needs to explore new states to discover potentially rewarding paths and at the same time, it has to exploit its current knowledge to get high short-term rewards. 

This is also known as $\varepsilon$-greedy policy. Here, we randomly set a probability ($\varepsilon$) and this is the probability with which the agent explores states randomn states, else, it exploits the policy/knowledge it has previously learnt. As we keep learning, gradually decreasing $\varepsilon$ is a good practice. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [23]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=11 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

**_position_**: 

- This is a matrix of size **_grid_size x grid_size_**. 
- It helps us define the rat's position and also helps us decide if certain moves are not allowed. 
- The default value for each cell is 0. However, the rat has a visibility of 2 cells and we need to define the boundaries of the grid so as to prevent the rat from leaving the grid. Therefore a +4 has been added to the grid_size as padding.
- The cell in which the rat is present has a value of 1. 

**_board_**:

- This matrix is also of size **_grid_size x grid_size_**.
- It helps us understand the environment of the rat, i.e. the rewards in each cell - `0.5` for cheese and `-1` for poison. 
- The value of a cell is set to zero after the rat has explored it. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # Selecting a random action
        act = np.random.randint(0, self.n_action)
        return act

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        
        state = env.reset()
        
        #Assuming that the games will end.
        game_over = False
        
        #Initializing counters
        win = 0
        lose = 0
        
        while not game_over:
            
            action = agent.learned_act(state)
            
            prev_state = state
            state, reward, game_over = env.act(action)
            
            if reward > 0:
                win += reward
            if reward < 0:
                lose -= reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()


test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.0/10.0. Average score (0.0)
Win/lose count 10.5/14.0. Average score (-1.75)
Win/lose count 11.5/20.0. Average score (-4.0)
Win/lose count 10.0/16.0. Average score (-4.5)
Win/lose count 10.0/12.0. Average score (-4.0)
Win/lose count 14.0/13.0. Average score (-3.1666666666666665)
Win/lose count 10.0/12.0. Average score (-3.0)
Win/lose count 9.0/12.0. Average score (-3.0)
Win/lose count 10.0/10.0. Average score (-2.6666666666666665)
Win/lose count 9.0/16.0. Average score (-3.1)
Final score: -3.1


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




## Answer

By definition, the $Q$ function is given by, $\mathcal{Q}^{\pi}(s,a)=E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s, a_{0}=a]$

By dropping the first term of the sum and using the linearity of the expectation, we have:
$$
\begin{align}
\mathcal{Q}^{\pi}(s,a)&=E_{p^{\pi}}\big[r(s_0,a_0)+\sum_{t=1}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a\big]\\
&=E_{p^{\pi}}\big[r(s_0,a_0)]+\gamma E_{p^{\pi}}\big[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a\big]\\
\end{align}
$$

Now, from the Markovian property we obtain:

$$
\begin{align}
\mathcal{Q}^{\pi}(s,a)&=E_{p^{\pi}}\big[r(s_{0},a_{0})\big]+\gamma E_{(s',a')~p(.|s,a)}\big[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1}|s_{1}=s',a_{1}=a')\big]\\
&=E_{(s',a')\sim p(.|s,a)}\big[r(s_{0},a_{0})\big]+\gamma E_{(s',a')\sim p(.|s,a)}\big[\sum_{t=0}^{\infty}
\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a'\big]\\
&=E_{(s',a')\sim p(.|s,a)}\big[r(s,a)+\gamma\mathcal{Q}^{\pi}(s',a')\big]\\
\end{align}
$$

If an optimal policy $\pi^{\ast}$ exists, then the optimal $Q$ function is defined as:

$$
\begin{align}
\mathcal{Q}^{\ast}(s,a)&=\text{max}_{\pi}\mathcal{Q}^{\pi}(s,a)\\
&=\text{max}_{\pi}E_{(s',a')\sim p(.|s,a)}\big[r(s,a)+\gamma\mathcal{Q}^{\pi}(s',a')\big]\\
&=E_{s'\sim\pi^{\ast}(.|s,a)}\big[r(s,a)+\gamma\mathcal{Q}^{\pi^{\ast}}(s',a')\big]\\
&=E_{s'\sim\pi^{\ast}(.|s,a)}\big[r(s,a)+\gamma \text{max}_{a'}\mathcal{Q}^{\ast}(s',a')\big]
\end{align}
$$

From these equations and due to the deterministic nature of $r$, the loss function that has to be minimized becomes:

$$
\begin{align}
\mathcal{L}(\theta)&=E_{s'\sim\pi^{\ast}(.|s,a)}\big[(\mathcal{Q}^{\ast}(s,a,\theta)-\mathcal{Q}(s,a,\theta))^{2}\big]\\
&=E_{s'\sim\pi^{\ast}(.|s,a)}\big[(r(s,a)+\gamma \text{max}_{a'}\mathcal{Q}^{\ast}(s',a',\theta)-\mathcal{Q}(s,a,\theta))^{2}\big]\\
&=E_{s'\sim\pi^{\ast}(.|s,a)}\big[||r+\gamma {max}_{a'}\mathcal{Q}(s',a',\theta)-\mathcal{Q}(s,a,\theta)||^{2}\big]
\end{align}
$$


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [10]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) >self.max_memory:
            self.memory.pop(0)
        

    def random_access(self):
        random_id = np.random.randint(len(self.memory))
        return self.memory[random_id]

***
The pipeline we will use for training is given below:

In [11]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [12]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
            
            input_states[i] = s_
            
            if game_over_:
                ######## FILL IN
                
                target_q[i,a_] = r_
                
            else:
                ######## FILL IN
                
                q_est = self.model.predict(np.expand_dims(n_s_, 0))
                target_q[i, a_] = r_ + self.discount * np.max(q_est)
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(32,activation ='relu')) 
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [16]:
epochs_train = 101
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 16)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train100.mp4'))

Epoch 000/101 | Loss 0.0068 | Win/lose count 2.0/2.0 (0.0)
Epoch 001/101 | Loss 0.0011 | Win/lose count 1.5/7.0 (-5.5)
Epoch 002/101 | Loss 0.0039 | Win/lose count 1.5/3.0 (-1.5)
Epoch 003/101 | Loss 0.0008 | Win/lose count 2.0/3.0 (-1.0)
Epoch 004/101 | Loss 0.0077 | Win/lose count 4.0/2.0 (2.0)
Epoch 005/101 | Loss 0.0200 | Win/lose count 3.0/10.0 (-7.0)
Epoch 006/101 | Loss 0.0023 | Win/lose count 3.5/3.0 (0.5)
Epoch 007/101 | Loss 0.0027 | Win/lose count 1.5/4.0 (-2.5)
Epoch 008/101 | Loss 0.0084 | Win/lose count 4.0/2.0 (2.0)
Epoch 009/101 | Loss 0.0204 | Win/lose count 4.5/1.0 (3.5)
Epoch 010/101 | Loss 0.1131 | Win/lose count 1.5/5.0 (-3.5)
Epoch 011/101 | Loss 0.0071 | Win/lose count 2.5/3.0 (-0.5)
Epoch 012/101 | Loss 0.0053 | Win/lose count 3.0/3.0 (0.0)
Epoch 013/101 | Loss 0.0093 | Win/lose count 3.0/1.0 (2.0)
Epoch 014/101 | Loss 0.0093 | Win/lose count 1.0/4.0 (-3.0)
Epoch 015/101 | Loss 0.0066 | Win/lose count 1.5/3.0 (-1.5)
Epoch 016/101 | Loss 0.0101 | Win/lose count 3

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [17]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        #Best model
        model.add(Conv2D(32,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(64,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [18]:
epochs_train = 101
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train100.mp4'))

Epoch 000/101 | Loss 0.0032 | Win/lose count 2.0/4.0 (-2.0)
Epoch 001/101 | Loss 0.0011 | Win/lose count 3.0/1.0 (2.0)
Epoch 002/101 | Loss 0.0089 | Win/lose count 2.5/5.0 (-2.5)
Epoch 003/101 | Loss 0.0084 | Win/lose count 3.5/1.0 (2.5)
Epoch 004/101 | Loss 0.0131 | Win/lose count 7.0/4.0 (3.0)
Epoch 005/101 | Loss 0.0086 | Win/lose count 3.0/4.0 (-1.0)
Epoch 006/101 | Loss 0.0078 | Win/lose count 4.0/3.0 (1.0)
Epoch 007/101 | Loss 0.0017 | Win/lose count 7.5/11.0 (-3.5)
Epoch 008/101 | Loss 0.0013 | Win/lose count 1.0/2.0 (-1.0)
Epoch 009/101 | Loss 0.0013 | Win/lose count 13.0/1.0 (12.0)
Epoch 010/101 | Loss 0.0046 | Win/lose count 1.5/3.0 (-1.5)
Epoch 011/101 | Loss 0.0103 | Win/lose count 7.0/5.0 (2.0)
Epoch 012/101 | Loss 0.0024 | Win/lose count 12.0/2.0 (10.0)
Epoch 013/101 | Loss 0.0014 | Win/lose count 7.5/3.0 (4.5)
Epoch 014/101 | Loss 0.0014 | Win/lose count 5.5/1.0 (4.5)
Epoch 015/101 | Loss 0.0009 | Win/lose count 16.0/4.0 (12.0)
Epoch 016/101 | Loss 0.0019 | Win/lose coun

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

### Low temperature (0.3)

In [24]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 3.5/0. Average score (3.5)
Win/lose count 0.5/0. Average score (2.0)
Win/lose count 10.0/0. Average score (4.666666666666667)
Win/lose count 4.0/0. Average score (4.5)
Win/lose count 15.0/0. Average score (6.6)
Win/lose count 0/0. Average score (5.5)
Win/lose count 8.5/0. Average score (5.928571428571429)
Win/lose count 5.5/0. Average score (5.875)
Win/lose count 1.5/0. Average score (5.388888888888889)
Win/lose count 9.0/0. Average score (5.75)
Win/lose count 7.0/0. Average score (5.863636363636363)
Final score: 5.863636363636363
Test of the FC
Win/lose count 1.0/1.0. Average score (0.0)
Win/lose count 0.5/1.0. Average score (-0.25)
Win/lose count 2.0/7.0. Average score (-1.8333333333333333)
Win/lose count 1.0/1.0. Average score (-1.375)
Win/lose count 0.5/0. Average score (-1.0)
Win/lose count 0/0. Average score (-0.8333333333333334)
Win/lose count 0/7.0. Average score (-1.7142857142857142)
Win/lose count 2.0/5.0. Average score (-1.875)
Win/lose count 2

In [25]:
HTML(display_videos('cnn_test10.mp4'))

In [26]:
HTML(display_videos('fc_test10.mp4'))

### High temperature (0.8)

In [27]:
env = Environment(grid_size=size, max_time=T,temperature=0.8)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test_ht')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test_ht')

Test of the CNN
Win/lose count 6.5/0. Average score (6.5)
Win/lose count 11.5/0. Average score (9.0)
Win/lose count 37.0/0. Average score (18.333333333333332)
Win/lose count 5.5/0. Average score (15.125)
Win/lose count 6.0/0. Average score (13.3)
Win/lose count 11.0/0. Average score (12.916666666666666)
Win/lose count 25.0/0. Average score (14.642857142857142)
Win/lose count 9.5/0. Average score (14.0)
Win/lose count 33.0/1.0. Average score (16.0)
Win/lose count 20.5/1.0. Average score (16.35)
Win/lose count 41.0/0. Average score (18.59090909090909)
Final score: 18.59090909090909
Test of the FC
Win/lose count 2.0/3.0. Average score (-1.0)
Win/lose count 5.0/3.0. Average score (0.5)
Win/lose count 4.0/0. Average score (1.6666666666666667)
Win/lose count 2.0/3.0. Average score (1.0)
Win/lose count 4.0/1.0. Average score (1.4)
Win/lose count 2.5/0. Average score (1.5833333333333333)
Win/lose count 4.0/3.0. Average score (1.5)
Win/lose count 7.5/2.0. Average score (2.0)
Win/lose count 3.0/

In [29]:
HTML(display_videos('fc_test_ht10.mp4'))

In [30]:
HTML(display_videos('cnn_test_ht10.mp4'))

**Fully Connected (FC):** The performance is poor compared to the CNN model. For a low temperature, it did not even explore. However, increasing the temperature made it perform a little better.

**CNN:** The CNN model's performance is much better than the FC model. It is able to reach more cheese even when it is far. Increasing the temperature also increased it's performance. 

The main **issue**, especially with the FC model, is that if the rat is stuck between two points, it often does not come out of that loop.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [31]:
def train_explore(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    # init. epsilon
    decreasing_factor = 0.9

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        # Update epsilon
        agent.set_epsilon(agent.epsilon * decreasing_factor)

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4 # Add border
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        # board on which one plays
        self.board = np.zeros((grid_size, grid_size))
        self.position = np.zeros((grid_size, grid_size))
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale = 16
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self, t):
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x, self.y, :] = 256
        b[-2:, :, :] = 0
        b[:, -2:, :] = 0
        b[:2, :, :]  = 0
        b[:, :2, :]  = 0
        b = cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        self.to_draw[t, :, :, :] = b

    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.position[self.x, self.y] = 1

        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: This action is not recognized')

        self.t += 1
        reward = (0 if not train else -self.malus_position[self.x, self.y]) + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        self.malus_position[self.x, self.y] += 0.1
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                                self.board.reshape(self.grid_size, self.grid_size, 1),
                                self.position.reshape(self.grid_size, self.grid_size, 1)),
                                axis=2)
        state = state[self.x-2:self.x+3, self.y-2:self.y+3, :]

        return state, reward, game_over

    def reset(self):
        """Resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus > 0] = 0
        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        
        self.board[self.x,self.y] = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = 0.1

        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                                self.board.reshape(self.grid_size, self.grid_size, 1),
                                self.position.reshape(self.grid_size, self.grid_size, 1)),
                                axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]

        return state

In [32]:
# Training
epochs_train = 101
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore100.mp4'))

Epoch 000/101 | Loss 0.0190 | Win/lose count 13.0/53.30000000000007 (-40.30000000000007)
Epoch 001/101 | Loss 0.0180 | Win/lose count 11.5/48.40000000000005 (-36.90000000000005)
Epoch 002/101 | Loss 0.0244 | Win/lose count 17.0/26.400000000000006 (-9.400000000000006)
Epoch 003/101 | Loss 0.0278 | Win/lose count 23.0/28.80000000000002 (-5.8000000000000185)
Epoch 004/101 | Loss 0.0228 | Win/lose count 21.5/20.399999999999984 (1.1000000000000156)
Epoch 005/101 | Loss 0.0066 | Win/lose count 21.5/20.099999999999994 (1.4000000000000057)
Epoch 006/101 | Loss 0.0204 | Win/lose count 14.5/18.19999999999998 (-3.6999999999999815)
Epoch 007/101 | Loss 0.0116 | Win/lose count 26.5/15.799999999999983 (10.700000000000017)
Epoch 008/101 | Loss 0.0108 | Win/lose count 17.5/17.800000000000004 (-0.30000000000000426)
Epoch 009/101 | Loss 0.0133 | Win/lose count 22.0/13.599999999999984 (8.400000000000016)
Epoch 010/101 | Loss 0.0125 | Win/lose count 24.5/15.099999999999978 (9.400000000000022)
Epoch 011/10

Epoch 093/101 | Loss 0.0328 | Win/lose count 26.5/19.39999999999999 (7.1000000000000085)
Epoch 094/101 | Loss 0.0489 | Win/lose count 22.5/18.69999999999998 (3.8000000000000185)
Epoch 095/101 | Loss 0.0371 | Win/lose count 20.5/24.30000000000003 (-3.800000000000029)
Epoch 096/101 | Loss 0.0390 | Win/lose count 25.5/17.899999999999988 (7.600000000000012)
Epoch 097/101 | Loss 0.0424 | Win/lose count 24.0/20.50000000000002 (3.4999999999999787)
Epoch 098/101 | Loss 0.0453 | Win/lose count 20.0/17.199999999999992 (2.800000000000008)
Epoch 099/101 | Loss 0.0703 | Win/lose count 24.0/18.699999999999992 (5.300000000000008)
Epoch 100/101 | Loss 0.0613 | Win/lose count 20.0/24.299999999999986 (-4.2999999999999865)


In [35]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 21.0/4.0. Average score (17.0)
Win/lose count 26.0/10.0. Average score (16.5)
Win/lose count 24.0/13.0. Average score (14.666666666666666)
Win/lose count 25.0/8.0. Average score (15.25)
Win/lose count 29.0/8.0. Average score (16.4)
Win/lose count 20.0/10.0. Average score (15.333333333333334)
Win/lose count 24.0/10.0. Average score (15.142857142857142)
Win/lose count 23.5/8.0. Average score (15.1875)
Win/lose count 26.0/13.0. Average score (14.944444444444445)
Win/lose count 27.5/11.0. Average score (15.1)
Win/lose count 24.0/10.0. Average score (15.0)
Final score: 15.0


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [41]:
class Mimic_Agent(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(Mimic_Agent, self).__init__(epsilon = epsilon)
        self.discount = 0.99
        self.grid_size = grid_size
        self.n_state = n_state
        self.memory = Memory(memory_size)
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_[:,:,1:], a_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, a_  = self.memory.random_access()
            
            target_q[i, a_] = 5
            input_states[i] = s_
        
        l = self.model.train_on_batch(input_states, target_q)
        
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
        
class CNN_Mimic_Agent(Mimic_Agent):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(CNN_Mimic_Agent, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        
# Training the Mimic Agent
def train_mimic_agent(ref, agent,env,epoch,epsilon=0.1, decay=0.99, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon(epsilon)

        while not game_over:
            # The agent performs an action
            action = ref.act(state, train=False)

            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [45]:
epochs_train = 101
original_agent = agent
mimic_agent = CNN_Mimic_Agent(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=2)
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
train_mimic_agent(original_agent, mimic_agent, env, epochs_train, prefix='mimic_agent')

Epoch 000/101 | Loss 2.1870 | Win/lose count 20.5/23.400000000000002 (-2.900000000000002)
Epoch 001/101 | Loss 2.4394 | Win/lose count 21.5/17.299999999999983 (4.200000000000017)
Epoch 002/101 | Loss 2.2297 | Win/lose count 27.0/17.69999999999999 (9.300000000000011)
Epoch 003/101 | Loss 2.0244 | Win/lose count 22.0/19.4 (2.6000000000000014)
Epoch 004/101 | Loss 1.8115 | Win/lose count 23.5/21.90000000000001 (1.5999999999999908)
Epoch 005/101 | Loss 2.0974 | Win/lose count 25.5/18.29999999999999 (7.20000000000001)
Epoch 006/101 | Loss 2.3537 | Win/lose count 24.0/20.299999999999997 (3.700000000000003)
Epoch 007/101 | Loss 2.5586 | Win/lose count 24.0/23.199999999999992 (0.8000000000000078)
Epoch 008/101 | Loss 2.7114 | Win/lose count 26.5/17.299999999999986 (9.200000000000014)
Epoch 009/101 | Loss 2.7604 | Win/lose count 27.5/15.599999999999987 (11.900000000000013)
Epoch 010/101 | Loss 3.1048 | Win/lose count 25.0/19.499999999999986 (5.500000000000014)
Epoch 011/101 | Loss 2.8038 | Win/

Epoch 094/101 | Loss 2.6988 | Win/lose count 17.0/18.39999999999998 (-1.3999999999999808)
Epoch 095/101 | Loss 2.3357 | Win/lose count 21.0/17.39999999999998 (3.600000000000019)
Epoch 096/101 | Loss 2.2757 | Win/lose count 26.5/20.099999999999998 (6.400000000000002)
Epoch 097/101 | Loss 2.2490 | Win/lose count 19.0/18.39999999999999 (0.6000000000000085)
Epoch 098/101 | Loss 2.2388 | Win/lose count 26.0/15.79999999999999 (10.20000000000001)
Epoch 099/101 | Loss 2.3507 | Win/lose count 22.0/18.699999999999992 (3.300000000000008)
Epoch 100/101 | Loss 2.9098 | Win/lose count 30.0/12.699999999999978 (17.300000000000022)


In [46]:
HTML(display_videos('mimic_agent100.mp4'))

**Comments:** 
- By mimicking the original agent, the performance is more or less the same. 
- However, training time reduces significantly. 

***